# Second Administrative Divisions of Countries

**[Work in progress]**

This notebook creates a .csv file with second administrative divisions (Counties in the US) for ingestion into the Knowledge Graph.

Data source: [GeoNames.org](https://download.geonames.org/export/dump/)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_HOME = Path(os.getenv('NEO4J_HOME'))
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-4af96121-2328-4e2f-ba60-6d8b728a26d5/installation-4.0.3


### Create admin2

In [4]:
admin2_url = 'https://download.geonames.org/export/dump/admin2Codes.txt'

In [5]:
names = ['code', 'name', 'name_ascii', 'geonameid']

In [6]:
admin2 = pd.read_csv(admin2_url, sep='\t', dtype='str', names=names)
admin2 = admin2[['code', 'name_ascii']]

### Standardize column names for Knowlege Graph
* id: unique identifier for country
* name: name of node
* parent_id: unique identifier for continent
* properties: camelCase

In [7]:
admin2.rename(columns={'code': 'id'}, inplace=True) # standard id column to link nodes
admin2.rename(columns={'name_ascii': 'name'}, inplace=True)
admin2['parentId'] = admin2['id'].str.rsplit('.', 1, expand=True)[0]

### Example

In [8]:
admin2.query("name == 'San Diego County'")

,id,name,parentId
42151,US.CA.073,San Diego County,US.CA


### Export a minimum subset for now

In [9]:
admin2 = admin2[['id', 'name', 'parentId']]
admin2 = admin2.fillna('')

In [10]:
admin2.head()

,id,name,parentId
0,AE.01.101,Abu Dhabi Municipality,AE.01
1,AE.01.102,Al Ain Municipality,AE.01
2,AE.01.103,Al Dhafra,AE.01
3,AE.04.701,Al Fujairah Municipality,AE.04
4,AE.04.702,Dibba Al Fujairah Municipality,AE.04


In [11]:
admin2.to_csv(NEO4J_HOME / "import/00g-GeoNamesAdmin2.csv", index=False)